# Predictve Safety Filter Benchmark

Load necessary packages, make sure to install `tinympc` ([README.md](../README.md))

In [3]:
import tinympc
import os
import numpy as np

In [4]:
path_to_root = os.getcwd()
print(path_to_root)

/home/khai/SSD/Code/mcu-solver-benchmarks/safety_filter


## Double Integrator System

In [5]:
NSTATES = 6
NINPUTS = 3
NHORIZON = 30
NTOTAL = 201

# Double-integrator dynamics
h = 0.05 #20 Hz
temp_n = int(NSTATES/2)
# Adyn = [I(temp_n) h*I(temp_n); zeros(temp_n,temp_n) I(temp_n)]
# Bdyn = [0.5*h*h*I(temp_n); h*I(temp_n)];
Adyn = np.block([[np.eye(temp_n), h*np.eye(temp_n)], [np.zeros((temp_n,temp_n)), np.eye(temp_n)]])
Bdyn = np.block([[0.5*h*h*np.eye(temp_n)], [h*np.eye(temp_n)]])

## MPC Formulation for OSQP

## Code Generation

We are done with the dynamics and LQR controller. Now, let's define the class and compile original TinyMPC code to get a generic shared/dynamic library


**PLEASE CHANGE `tinympc_python_dir` TO YOUR ABSOLUTE PATH**

In [6]:
tinympc_python_dir = path_to_root + "/../tinympc-python"  # Your absolute path to the tinympc-python directory
tinympc_dir = tinympc_python_dir + "/tinympc/TinyMPC"  # Path to the TinyMPC directory (C code)

tinympc_prob = tinympc.TinyMPC()
tinympc_prob.compile_lib(tinympc_dir)  # Compile the library (or use the binary provided)

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.




-- Configuring done (0.0s)
-- Generating done (0.1s)
-- Build files have been written to: /home/khai/SSD/Code/mcu-solver-benchmarks/tinympc-python/tinympc/TinyMPC/build
[ 18%] Built target tinympc
[ 37%] Built target tinympcShared
[ 50%] Built target quadrotor_tracking
[ 62%] Built target quadrotor_hovering
[ 75%] Built target codegen_random
[ 87%] Built target codegen_cartpole
[100%] Built target test1


True

In [7]:
os_ext = ".so"  # CHANGE THIS BASED ON YOUR OS
lib_dir = tinympc_dir + "/build/src/tinympc/libtinympcShared" + os_ext  # Path to the compiled library
tinympc_prob.load_lib(lib_dir)  # Load the library

## We generate a bunch of benchmark here with varying dimensions

Here we setup problem data and settings for the predictive safety filter with TinyMPC. We will put bounds on the state and input.

In [8]:
A = Adyn.transpose().reshape((NSTATES * NSTATES)).tolist() # col-major order list
B = Bdyn.transpose().reshape((NSTATES * NINPUTS)).tolist() # col-major order list
Q = np.zeros(NSTATES).tolist()  # diagonal of state cost -- DON'T NEED FOR SAFETY FILTER
R = np.ones(NINPUTS).tolist()  # diagonal of input cost
rho = 1e2  # ADMM penalty parameter

x_min = [-10.0] * NSTATES * NHORIZON           # state constraints 
x_max = [10.] * NSTATES * NHORIZON             # state constraints
u_min = [-3] * NINPUTS * (NHORIZON - 1)      # input constraints
u_max = [3] * NINPUTS * (NHORIZON - 1)   # input constraints

abs_pri_tol = 1.0e-3    # absolute primal tolerance
abs_dual_tol = 1.0e-3   # absolute dual tolerance
max_iter = 100          # maximum number of iterations
check_termination = 1   # whether to check termination and period

# Setup problem data
tinympc_prob.setup(NSTATES, NINPUTS, NHORIZON, A, B, Q, R, x_min, x_max, u_min, u_max, rho, abs_pri_tol, abs_dual_tol, max_iter, check_termination)

True

Load the generic shared/dynamic library. **You may want to change the extension of the library based on your OS -- Linux: .so, Mac: .dylib, Windows: .dll**

After define the problem, we generate the tailored code with above data. 

**Here we compile it for interactive Python script but you can use it directly for your applications/systems**

In [9]:
output_dir = path_to_root + "/tinympc_generated"  # Path to the generated code
tinympc_prob.tiny_codegen(tinympc_dir, output_dir)  
# You may want to check if Kinf in generated_code follows the same pattern as previous K in LQR, otherwise something is wrong
tinympc_prob.compile_lib(output_dir)

A = [   1,    0,    0, 0.05,    0,    0]
[   0,    1,    0,    0, 0.05,    0]
[   0,    0,    1,    0,    0, 0.05]
[   0,    0,    0,    1,    0,    0]
[   0,    0,    0,    0,    1,    0]
[   0,    0,    0,    0,    0,    1]
B = [0.00125,       0,       0]
[      0, 0.00125,       0]
[      0,       0, 0.00125]
[   0.05,       0,       0]
[      0,    0.05,       0]
[      0,       0,    0.05]
Q = [100,   0,   0,   0,   0,   0]
[  0, 100,   0,   0,   0,   0]
[  0,   0, 100,   0,   0,   0]
[  0,   0,   0, 100,   0,   0]
[  0,   0,   0,   0, 100,   0]
[  0,   0,   0,   0,   0, 100]
R = [101,   0,   0]
[  0, 101,   0]
[  0,   0, 101]
rho = 100
Kinf converged after 128 iterations
Precomputing finished
Kinf = [0.9529,      0,      0,  1.677,      0,      0]
[     0, 0.9529,      0,      0,  1.677,      0]
[     0,      0, 0.9529,      0,      0,  1.677]
Pinf = [3520,    0,    0, 2010,    0,    0]
[   0, 3520,    0,    0, 2010,    0]
[   0,    0, 3520,    0,    0, 2010]
[2010,    0,    0, 3

CMake Deprecation Warning at CMakeLists.txt:5 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.




[ 11%] Building CXX object tinympc/CMakeFiles/tinympc.dir/admm.cpp.o
[ 22%] Linking CXX static library libtinympc.a
[ 22%] Built target tinympc
[ 33%] Building CXX object tinympc/CMakeFiles/tinympcShared.dir/admm.cpp.o
[ 44%] Building CXX object tinympc/CMakeFiles/tinympcShared.dir/tiny_wrapper.cpp.o
[ 55%] Building CXX object tinympc/CMakeFiles/tinympcShared.dir/__/src/tiny_data_workspace.cpp.o
[ 66%] Linking CXX shared library libtinympcShared.so
[ 66%] Built target tinympcShared
[ 77%] Building CXX object src/CMakeFiles/tiny_main.dir/tiny_main.cpp.o
[ 88%] Building CXX object src/CMakeFiles/tiny_main.dir/tiny_data_workspace.cpp.o
[100%] Linking CXX executable tiny_main
[100%] Built target tiny_main


True

## We then copy generated workspace from `tinympc_generated` to `teensy_tinympc` dir for each benchmark.

In [10]:
import osqp
from scipy import sparse

Ad = sparse.csc_matrix(Adyn)
Bd = sparse.csc_matrix(Bdyn)

x0 = np.ones(NSTATES)*0.5

Xref = np.zeros((NSTATES, NTOTAL))
for k in range(NTOTAL):
    Xref[0:3,k] = np.sin(1*k)*2*np.ones(temp_n)
Uref = np.ones((NINPUTS, NTOTAL-1))*3

Qnp = np.zeros((NSTATES, NSTATES))
Rnp = 1e2*np.ones((NINPUTS, NINPUTS))

# Cast MPC problem to a QP: x = (x(0),x(1),...,x(N),u(0),...,u(N-1))
# - quadratic objective
P = sparse.block_diag([sparse.kron(sparse.eye(NHORIZON), Qnp),
                       sparse.kron(sparse.eye(NHORIZON-1), Rnp)], format='csc')
# - linear objective
q = np.hstack([np.zeros((NHORIZON)*NSTATES), np.hstack([-Rnp@Uref[:,i] for i in range(NHORIZON-1)])])
# - linear dynamics
Ax = sparse.kron(sparse.eye(NHORIZON),-sparse.eye(NSTATES)) + sparse.kron(sparse.eye(NHORIZON, k=-1), Ad)
Bu = sparse.kron(sparse.vstack([sparse.csc_matrix((1, NHORIZON-1)), sparse.eye(NHORIZON-1)]), Bd)
Aeq = sparse.hstack([Ax, Bu])
leq = np.hstack([-x0, np.zeros((NHORIZON-1)*NSTATES)])
ueq = leq

# - input and state constraints
xmin = x_min[0:NSTATES]
xmax = x_max[0:NSTATES]
umin = u_min[0:NINPUTS]
umax = u_max[0:NINPUTS]
Aineq = sparse.eye((NHORIZON)*NSTATES + (NHORIZON-1)*NINPUTS)
lineq = np.hstack([np.kron(np.ones(NHORIZON), xmin), np.kron(np.ones(NHORIZON-1), umin)])
uineq = np.hstack([np.kron(np.ones(NHORIZON), xmax), np.kron(np.ones(NHORIZON-1), umax)])
# - OSQP constraints
A = sparse.vstack([Aeq, Aineq], format='csc')
l = np.hstack([leq, lineq])
u = np.hstack([ueq, uineq])

# Create an OSQP object
osqp_prob = osqp.OSQP()

# Setup workspace and change alpha parameter
osqp_prob.setup(P, q, A, l, u, alpha=1.0, scaling=0, check_termination=check_termination, eps_abs=abs_pri_tol, eps_rel=1e-3, eps_prim_inf=1e-4, eps_dual_inf=1e-4, max_iter=max_iter, polish=False, rho=rho, adaptive_rho=False, warm_start=True)
# prob.setup(P, q, A, l, u, warm_starting=True, polish=True)

res = osqp_prob.solve()
x = res.x[0:NSTATES*NHORIZON]
u = res.x[NSTATES*NHORIZON:]
print(x)
print(u)

# Generate C code
# fmt: off
osqp_prob.codegen(
    path_to_root+'/osqp_generated',   # Output folder for auto-generated code
    prefix='osqp_data_',         # Prefix for filenames and C variables; useful if generating multiple problems
    force_rewrite=True,        # Force rewrite if output folder exists?
    parameters='vectors',      # What do we wish to update in the generated code?
                                # One of 'vectors' (allowing update of q/l/u through prob.update_data_vec)
                                # or 'matrices' (allowing update of P/A/q/l/u
                                # through prob.update_data_vec or prob.update_data_mat)
    use_float=True,
    printing_enable=False,     # Enable solver printing?
    profiling_enable=False,    # Enable solver profiling?
    interrupt_enable=False,    # Enable user interrupt (Ctrl-C)?
    include_codegen_src=True,  # Include headers/sources/Makefile in the output folder,
                                # creating a self-contained compilable folder?
    extension_name='pyosqp',   # Name of the generated python extension; generates a setup.py; Set None to skip
    compile=False,             # Compile the above python extension into an importable module
                                # (allowing "import pyosqp")?
)
# fmt: on


-----------------------------------------------------------------
           OSQP v1.0.0.beta0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 267, constraints m = 447
          nnz(P) + nnz(A) = 1056
settings: algebra = Built-in,
          linear system solver = QDLDL v0.1.6,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e+02 ,
          sigma = 1.00e-06, alpha = 1.00, max_iter = 100
          check_termination: on (interval 1),
          time_limit: 1.00e+10 sec,
          scaling: off, scaled_termination: off
          warm starting: on, polishing: off, 
iter   objective    prim res   dual res   rho        time
   1  -1.0297e+05   5.37e-01   5.00e+04   1.00e+02   2.92e-04s
  13  -1.1745e+05   2.61e-03   1.44e-01   1.00e+0

'/home/khai/SSD/Code/mcu-solver-benchmarks/safety_filter/osqp_generated/'